# import

In [76]:
import pandas as pd
import ast
import numpy as np
from tqdm import tqdm


In [3]:
import torch
import torch.nn as nn
from collections import Counter
import ast
from torch.utils.data import Dataset, DataLoader
import torch
from torch.utils.data import DataLoader, TensorDataset

In [4]:
device = torch.device('mps')

In [5]:
df1 = pd.read_csv('./data/userList (10001~20001).csv')
df2 = pd.read_csv('./data/userList(20001~30000).csv')
df3 = pd.read_csv('./data/userList(30000~40001).csv')
df4 = pd.read_csv('./data/userList(40001~50001).csv')
df5 = pd.read_csv('./data/userList.csv')


In [6]:
# DataFrame들을 리스트로 결합
#dfs = [df1, df2, df3, df4, df5]
dfs = [df1, df2, df3, df4]

# concat 함수를 사용하여 DataFrame들을 하나로 합침
combined_df = pd.concat(dfs)



In [7]:
combined_df.reset_index(drop=True, inplace=True)

In [8]:
combined_df.columns

Index(['id', 'tier', 'rank', 'rightCnt', 'wrongCnt', 'time out',
       'memory exceed', 'print exceed', 'runTime error', 'compile error',
       'solvedProblemList', 'triedNotsolvedList'],
      dtype='object')

In [9]:
df5.columns = combined_df.columns

In [10]:
df5.isna().sum()

id                    111
tier                   56
rank                  111
rightCnt              111
wrongCnt              111
time out              111
memory exceed         111
print exceed          111
runTime error         111
compile error         111
solvedProblemList     111
triedNotsolvedList    165
dtype: int64

In [11]:
df5.dropna(inplace=True)

In [12]:
combined_df = pd.concat([combined_df, df5])

In [13]:
combined_df.reset_index(drop=True, inplace=True)

In [14]:
col = 'rank	rightCnt	wrongCnt	time out	memory exceed	print exceed	runTime error	compile error	solvedProblemList	triedNotsolvedList'
col = col.split('\t')

In [15]:
def check_no_list(c, df):
    temp = df[df[c] == '[]']
    return temp

In [16]:
is_null = pd.DataFrame()

In [17]:
for i in range(len(col)):
    temp = check_no_list(col[i], combined_df)
    is_null = pd.concat([temp, is_null])

In [18]:
is_null

,id,tier,rank,rightCnt,wrongCnt,time out,memory exceed,print exceed,runTime error,compile error,solvedProblemList,triedNotsolvedList
16,lake041,16,['9667'],['808'],['358'],['221'],['46'],['31'],['151'],['19'],"['1000', '1001', '1003', '1008', '1012', '1013...",[]
19,ghwns9652,15,['9944'],['382'],['114'],['49'],['14'],['7'],['28'],['10'],"['1000', '1001', '1002', '1003', '1004', '1005...",[]
23,1k2ms,15,['9838'],['374'],['156'],['59'],['28'],['3'],['36'],['17'],"['1000', '1001', '1003', '1008', '1009', '1010...",[]
52,doitforme,12,['9865'],['506'],['296'],['24'],['2'],['5'],['77'],['186'],"['1000', '1001', '1008', '1009', '1010', '1032...",[]
109,limce21,15,['10151'],['526'],['309'],['123'],['30'],['6'],['133'],['17'],"['1000', '1001', '1002', '1003', '1008', '1010...",[]
...,...,...,...,...,...,...,...,...,...,...,...,...
47594,blue23,0,['9727'],['353'],['0'],['533'],['372'],['2'],['38'],['26'],[],[]
47646,polestar7,0,['9786'],['352'],['1'],['483'],['359'],['52'],['17'],['3'],[],[]
47698,visionary,17,['9840'],['351'],['1'],['509'],['368'],['19'],['5'],['1'],[],[]
53127,fill_blank,12,['55381'],['108'],['1'],['158'],['109'],none,['12'],['3'],[],[]


In [19]:
# is_null 데이터프레임에 있는 행의 인덱스를 가져옴
drop_indices = is_null.index

# 이 인덱스를 사용하여 combined_df에서 해당 행을 제거
combined_df_dropped = combined_df.drop(drop_indices)


In [20]:
df = combined_df_dropped.reset_index(drop=True)

In [21]:
df

,id,tier,rank,rightCnt,wrongCnt,time out,memory exceed,print exceed,runTime error,compile error,solvedProblemList,triedNotsolvedList
0,jh5154,15,['10033'],['546'],['252'],['76'],['2'],['7'],['42'],['33'],"['1000', '1001', '1003', '1008', '1012', '1018...","['1722', '1725', '1916', '11279', '12902', '13..."
1,baefr,15,['10034'],['515'],['235'],['100'],['34'],['5'],['53'],['26'],"['1000', '1001', '1002', '1008', '1010', '1011...","['1062', '2343', '2615', '2931', '2954', '3025..."
2,nacean,15,['10035'],['423'],['339'],['107'],['20'],['6'],['36'],['40'],"['1000', '1001', '1003', '1008', '1009', '1010...","['1038', '1039', '1937', '1981', '2407', '2533..."
3,klxex,15,['10036'],['451'],['271'],['88'],['35'],['4'],['83'],['45'],"['1003', '1005', '1010', '1012', '1018', '1021...","['1253', '1509', '1774', '2133', '2473', '7662..."
4,qortndud97,15,['10037'],['570'],['250'],['57'],['32'],['14'],['35'],['38'],"['1000', '1001', '1005', '1008', '1009', '1010...","['1153', '1701', '1777', '1799', '1865', '2076..."
...,...,...,...,...,...,...,...,...,...,...,...,...
84114,kjsmel,5,['100269'],['58'],['1'],['116'],none,['22'],['7'],['3'],"['1000', '1001', '1008', '1065', '1110', '1152...",['1011']
84115,limy0213,5,['100269'],['58'],['1'],none,none,none,['9'],['16'],"['1000', '1001', '1008', '1026', '1094', '1100...",['10172']
84116,palzmnbvcx,5,['100269'],['58'],['4'],['116'],['60'],none,['6'],['2'],"['1000', '1001', '1008', '1110', '1330', '1427...","['1181', '1929', '10814', '10989']"
84117,planet4869,6,['100269'],['58'],['4'],['116'],none,none,['9'],['1'],"['1000', '1001', '1010', '1094', '1149', '1152...","['1008', '1475', '1929', '2805']"


In [22]:
df[df['solvedProblemList'] == '[]']

,id,tier,rank,rightCnt,wrongCnt,time out,memory exceed,print exceed,runTime error,compile error,solvedProblemList,triedNotsolvedList


# tier 0인거 확인

In [23]:
df[df['tier'] == 0]

,id,tier,rank,rightCnt,wrongCnt,time out,memory exceed,print exceed,runTime error,compile error,solvedProblemList,triedNotsolvedList
429,davincicoding,0,['10424'],['649'],['260'],['98'],['60'],['11'],['109'],['23'],"['1000', '1001', '1002', '1005', '1008', '1010...",['21759']
534,hosung0821,0,['10619'],['368'],['68'],['22'],['18'],['2'],['47'],['27'],"['1000', '1001', '1003', '1004', '1005', '1007...","['2492', '2943', '6198', '6986', '7579', '8907..."
535,jackae98,0,['10620'],['354'],['152'],['20'],['6'],['2'],['11'],['20'],"['1000', '1001', '1002', '1003', '1004', '1008...","['1005', '1520', '1640', '1747', '1902', '1931..."
536,npqr11,0,['10621'],['381'],['127'],['36'],['7'],['2'],['17'],['15'],"['1000', '1001', '1002', '1003', '1004', '1008...","['1753', '1904', '1915', '1920', '1932', '1937..."
824,jmcabcd,0,['10941'],['360'],['211'],['44'],['5'],['6'],['33'],['12'],"['1000', '1001', '1002', '1003', '1005', '1008...","['1700', '1987', '2931']"
...,...,...,...,...,...,...,...,...,...,...,...,...
34024,platform,0,['49214'],['121'],['1'],['18'],['382'],['133'],['180'],['30'],"['1000', '1012', '1051', '1068', '1085', '1100...","['1021', '1260', '1654', '1717', '1931', '1976..."
34025,simons,0,['49214'],['121'],['2'],['382'],['179'],['134'],['44'],['10'],"['1035', '1156', '1187', '1396', '1786', '1804...","['14778', '17020']"
34086,marksung,0,['49286'],['121'],['18'],['455'],none,['171'],['2'],['182'],"['1000', '1001', '1006', '1019', '1181', '1244...","['1008', '1016', '1075', '1271', '1809', '2231..."
34087,areumchoi19,0,['49289'],['121'],['5'],['457'],['257'],['113'],['38'],['2'],"['1000', '1001', '1002', '1003', '1004', '1008...","['1005', '1325', '1389', '1463', '9466']"


- 순위에 비해서 tier가 0인건 말이안됨
- 이상치라 판단하고 드롭

In [24]:
tier_0 = df[df['tier'] == 0].index
df.drop(tier_0, inplace=True)

# 리스트 안에 있는 숫자 밖으로 빼내기

In [25]:
# 문자열을 리스트로 변환하는 함수 정의
def string_to_list(s):
    try:
        return ast.literal_eval(s)
    except ValueError:
        return []  # 또는 적절한 기본값 반환

In [26]:
# 리스트로 꺼내도 될 칼럼 정의
li='rank	rightCnt	wrongCnt	time out	memory exceed	print exceed	runTime error	compile error'
col = li.split('\t')

# 리스트 안 숫자 꺼내는 함수
def turn_num(s):
    cs = s.apply(string_to_list).explode()
    return cs

# 데이터프레임에 모두 적용
for i in range(len(col)):
    df[col[i]] = turn_num(df[col[i]])


In [27]:
# 해당 칼럼들은 모두 숫자형이기 때문에 int로 변환
# 근데 nan 값을 먼저 확인해보자
df.isna().sum()

id                        0
tier                      0
rank                      0
rightCnt                  0
wrongCnt                  8
time out               1589
memory exceed         18768
print exceed          10654
runTime error           655
compile error          1254
solvedProblemList         0
triedNotsolvedList        0
dtype: int64

In [28]:
# 일단 nan값 많아서 드롭보다는 0으로 변환하기

# 각 칼럼에서 정수로 변환할 수 없는 값을 찾기
for column in col:
    # to_numeric을 사용하여 변환할 수 없는 값을 NaN으로 변경
    df[column] = pd.to_numeric(df[column], errors='coerce')

    # NaN 값이 있는 행 찾기
    invalid_rows = df[df[column].isnull()]

    # 결과 확인
    if not invalid_rows.empty:
        print(f"Column '{column}' contains non-integer values in the following rows:")

Column 'wrongCnt' contains non-integer values in the following rows:
Column 'time out' contains non-integer values in the following rows:
Column 'memory exceed' contains non-integer values in the following rows:
Column 'print exceed' contains non-integer values in the following rows:
Column 'runTime error' contains non-integer values in the following rows:
Column 'compile error' contains non-integer values in the following rows:


In [29]:
invalid_rows

,id,tier,rank,rightCnt,wrongCnt,time out,memory exceed,print exceed,runTime error,compile error,solvedProblemList,triedNotsolvedList
17110,krh512,14,30149,179,8.0,570.0,201.0,2.0,187.0,NaN,"['1003', '1010', '1012', '1018', '1025', '1026...","['1002', '1107', '1339', '1342', '1525', '1682..."
17191,climbcode,10,30230,178,1.0,309.0,179.0,1.0,70.0,NaN,"['1000', '1001', '1008', '1010', '1018', '1085...",['1002']
17431,ho991227,7,30515,177,2.0,7.0,NaN,213.0,1.0,NaN,"['1000', '1001', '1003', '1008', '1018', '1085...","['1271', '4673', '8437', '14173', '15051', '24..."
17480,nyn2265,12,30562,177,1.0,8.0,433.0,218.0,1.0,NaN,"['1000', '1001', '1008', '1009', '1021', '1068...","['2357', '4134', '6416', '9489', '17626', '183..."
17628,chutzpah,8,30722,176,2.0,218.0,186.0,4.0,15.0,NaN,"['1000', '1001', '1008', '1085', '1110', '1152...","['2231', '11650']"
...,...,...,...,...,...,...,...,...,...,...,...,...
83940,saladfingers,6,99970,58,1.0,91.0,NaN,NaN,6.0,NaN,"['1000', '1001', '1008', '1085', '1152', '1157...",['1920']
83965,mapside,6,100020,58,1.0,96.0,NaN,1.0,24.0,NaN,"['1000', '1001', '1008', '1011', '1065', '1110...",['1978']
83974,esthern,6,100043,58,1.0,98.0,NaN,NaN,24.0,NaN,"['1065', '1182', '1408', '1966', '1977', '1978...",['2294']
83984,jiyun_choi,5,100059,58,1.0,99.0,NaN,NaN,20.0,NaN,"['1000', '1001', '1008', '1065', '1110', '1152...",['2869']


In [30]:
df[col] = df[col].apply(lambda x: x.fillna(0).astype(int))

# 인덱스 초기화하기

In [31]:
df.reset_index(drop=True, inplace=True)

# 데이터 분포 확인

In [32]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84022 entries, 0 to 84021
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  84022 non-null  object
 1   tier                84022 non-null  object
 2   rank                84022 non-null  int64 
 3   rightCnt            84022 non-null  int64 
 4   wrongCnt            84022 non-null  int64 
 5   time out            84022 non-null  int64 
 6   memory exceed       84022 non-null  int64 
 7   print exceed        84022 non-null  int64 
 8   runTime error       84022 non-null  int64 
 9   compile error       84022 non-null  int64 
 10  solvedProblemList   84022 non-null  object
 11  triedNotsolvedList  84022 non-null  object
dtypes: int64(8), object(4)
memory usage: 7.7+ MB


In [33]:
# 숫자형 변환
df['tier'] = df['tier'].astype(int)

In [34]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 84022 entries, 0 to 84021
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   id                  84022 non-null  object
 1   tier                84022 non-null  int64 
 2   rank                84022 non-null  int64 
 3   rightCnt            84022 non-null  int64 
 4   wrongCnt            84022 non-null  int64 
 5   time out            84022 non-null  int64 
 6   memory exceed       84022 non-null  int64 
 7   print exceed        84022 non-null  int64 
 8   runTime error       84022 non-null  int64 
 9   compile error       84022 non-null  int64 
 10  solvedProblemList   84022 non-null  object
 11  triedNotsolvedList  84022 non-null  object
dtypes: int64(9), object(3)
memory usage: 7.7+ MB


In [35]:
# 숫자형만 확인
numeric_df = df.select_dtypes(include=['number'])

In [36]:
df.describe()

,tier,rank,rightCnt,wrongCnt,time out,memory exceed,print exceed,runTime error,compile error
count,84022.000000,84022.000000,84022.000000,84022.000000,84022.000000,84022.000000,84022.000000,84022.000000,84022.000000
mean,10.627193,48217.119528,228.861132,94.356026,229.842577,97.603687,30.938445,69.430732,33.924496
std,3.600479,28631.409370,321.696631,218.873024,212.019108,149.430191,65.702318,73.298060,54.174742
min,0.000000,55.000000,43.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,8.000000,23372.500000,83.000000,2.000000,79.000000,2.000000,2.000000,24.000000,6.000000
50%,10.000000,47598.000000,124.000000,6.000000,190.000000,57.000000,5.000000,51.000000,18.000000
75%,13.000000,72457.500000,265.000000,114.000000,313.000000,140.000000,23.000000,91.000000,40.000000
max,31.000000,127632.000000,12975.000000,7530.000000,4097.000000,3587.000000,2073.000000,1815.000000,1354.000000


- 모두 이상치 없음